In [17]:
import csv
import sklearn
import pandas as pd
from numpy import array
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [18]:
df = pd.read_csv('spam.csv', encoding='latin')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [19]:
df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
df = df.rename(columns={"v2" : "text", "v1":"label"})

In [20]:
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [21]:
# Import nltk packages and Punkt Tokenizer Models
# import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\divya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
df = df.replace(['ham','spam'],[0, 1]) 

In [23]:
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


## Removing all stopwords

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\divya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
#remove the punctuations and stopwords
import string
def text_process(text):
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    
    return " ".join(text)

In [26]:
df['text'] = df['text'].apply(text_process)

In [29]:
text = df['text']
label = df['label'].to_numpy()

## Converting words to vectors using TFIDF Vectorizer

In [30]:
#convert the text data into vectors
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import CountVectorizer

vectorizer = TfidfVectorizer()
# vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(df['text'])
vectors.shape

(5572, 9376)

In [31]:
# features = word_vectors
features = vectors
# fetures = features.toarray()
features = features.toarray().astype(float)
features.shape

(5572, 9376)

In [32]:
# #decision tree

# import numpy as np
# # from Node import Node

# class Node:
#     def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
#         self.feature = feature
#         self.threshold = threshold
#         self.left = left
#         self.right = right
#         self.value = value
        
#     def is_leaf(self):
#         return self.value is not None
  
  
# class DecisionTree:
#     #by default it will make root = NUll and assign max_depth and min_samples_split
#     def __init__(self, max_depth=10, min_samples_split=2):
#         self.max_depth = max_depth
#         self.min_samples_split = min_samples_split
#         self.root = None
        
#     # it will check whethr we should stop now or not 
#     # here if depth is reach our max or class split is divided into 1 or split samples are less min split
#     def searchComplete(self, depth):
#         if (depth >= self.max_depth or self.class_labels == 1 or self.n_samples < self.min_samples_split):
#             return True
#         return False
    
#     def entropy(self,y):
#         proportions = np.bincount(y) / len(y)
#         entropy = -np.sum([p * np.log2(p) for p in proportions if p > 0])
#         return entropy
        
# 	# make the tree split using threshold value
#     def createSplit(self, X , thresh):
#         print(X)
#         left_idx = np.argwhere(X <= thresh)
#         right_idx = np.argwhere(X > thresh)
        
#         # print(left_idx,right_idx)
#         return left_idx, right_idx
        
# 	# implementing the basic information gain formula finding the left subtree and right subtree and subtracting the child loss from its parent loss
#     def informationGain(self, X, y, thresh):
#         parentLoss = self.entropy(y)
#         left_idx, right_idx = self.createSplit(X, thresh)
#         n, n_left , n_right = len(y), len(left_idx), len(right_idx)

#         if n_left == 0 or n_right == 0:
#             return 0
#         childLoss = (n_left / n) * self.entropy(y[left_idx]) + (n_right/ n) * self.entropy(y[right_idx])
#         return parentLoss - childLoss

# 	# best function to make the split 
#     def bestSplit(self, X, y , features):
#         split = {'score' : -1 , 'feat': None, 'thresh': None}
       
#         for feat in features:
#             X_feat = X[feat]
#             thresholds = np.unique(X_feat)
#             for thresh in thresholds:
#                 score = self.informationGain(X_feat, y, thresh)
                
#                 if score > split['score']:
#                     split['score'] = score
#                     split['feat'] = feat
#                     split['thresh'] = thresh
#         return split['feat'], split['thresh']

        
#     # will have to build tree with given information of depth and training dataset
#     def buildTree(self, X, y, depth = 0):

#         self.n_samples, self.n_features = X.shape 
#         self.class_labels = len(np.unique(y)) 
            
#         # check whether we should stop or not
#         if self.searchComplete(depth):
#             most_common_Label = np.argmax(np.bincount(y))
#             return Node(value = most_common_Label)

#         # get best split
#         rnd_feats = np.random.choice(self.n_features, self.n_features, replace=False) # generates a new random sample from 1D array
#         best_feat, best_thresh = self.bestSplit(X, y, rnd_feats) # this function will return best feature and best threshold 
         
#         # grow child recursively
#         left_idx, right_idx = self.createSplit(X[best_feat], best_thresh)
#         left_child = self.buildTree(X[left_idx], y[left_idx], depth + 1)
#         right_child = self.buildTree(X[right_idx], y[right_idx], depth + 1)
#         return Node(best_feat, best_thresh, left_child, right_child)
        
#     def _traverse_tree(self, x, node):
#         if node.is_leaf():
#             return node.value
        
#         if x[node.feature] <= node.threshold:
#             return self._traverse_tree(x, node.left)
#         return self._traverse_tree(x, node.right)
    
#     # will call this function in our jupyter notebook during 
#     # building model instead of calling the prebuilt decision tree using sckit learn 
#     def fit(self, X, y):
#         self.root = self.buildTree(X,y)
#         return 0

#     def predict(self, X):
#         predictions = [self._traverse_tree(x, self.root) for x in X ]
#         return np.array(predictions)



In [33]:
import random
from tqdm import tqdm

In [34]:
class PSO:
    def __init__(self, particles, dimensions, lb, ub, itrations, c1, c2, w, xr, yr):
        self.xr = xr
        self.yr = yr
        self.lb = lb
        self.ub = ub
        self.p = particles
        self.d = dimensions
        self.n = itrations
        self.c1 = c1
        self.c2 = c2
        self.w = w
        self.fitness_value = np.zeros(particles) + np.inf  # p
        self.X = np.random.randint(0, 2, (particles, dimensions))  # pxd
        self.V = np.random.randint(0, 2, (particles, dimensions))  # pxd
        self.pbest = self.X  # pxd
        self.gbest = np.zeros(dimensions)  # d
        # ---initialiastion-------------------------
        min = np.inf
        for i in range(self.p):
            fit = self.fitness(self.X[i])
            if min > fit:
                min = fit
                self.gbest = self.X[i]
        # -------------------------------------------
        self.run()

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def count(self, x):
        c = np.sum(x)
        return c

    def fitness(self, x):
        x_train = []
        y_train = self.yr

        for i in range(self.d):
            if x[i] == 1:
                x_train.append(self.xr[:, i])

        x_train = np.array(x_train).reshape((self.xr.shape[0], -1))

        if x_train.shape[1] == 0:
            return 0

        classifier = DecisionTreeClassifier(max_depth = 3)
        classifier.fit(x_train, y_train)
        chk = classifier.predict(x_train)
        
        acc = ((chk == y_train).sum() * 1.0) / len(chk)
        return acc

    def run(self):
        for itr in tqdm(range(int(self.n))):
            for i in range(self.p):
                if self.fitness(self.X[i]) > self.fitness(self.pbest[i]):
                    self.pbest[i] = self.X[i]

                if self.fitness(self.X[i] > self.fitness(self.gbest)):
                    self.gbest = self.X[i]

                if (self.fitness(self.X[i]) == self.fitness(self.gbest)) and (self.count(self.X[i]) < self.count(self.gbest)):
                    self.gbest = self.X[i]

            r1 = random.uniform(0, 1)
            r2 = random.uniform(0, 1)
            for p in range(self.p):
                for q in range(self.d):
                    self.V[p][q] = (self.w * self.V[p][q] + self.c1 * r1 * (self.pbest[p][q] - self.X[p][q]) + self.c2 * r2 * (self.gbest[q] - self.X[p][q]))

                    if self.V[p][q] > self.ub:
                        self.V[p][q] = self.ub
                    if self.V[p][q] < self.lb:
                        self.V[p][q] = self.ub

                    if self.sigmoid(self.V[p][q]) > random.uniform(0, 1):
                        self.X[p][q] = 1
                    else:
                        self.X[p][q] = 0

In [37]:
p = 100 #number of particles
d = 1300 #total number of features
lb = 0 #min value after normalization
ub = 1 #max value after normalization
it = 30 #number of itrations
c1 = 1.5 # coefficient 1
c2 = 1.7 # coefficient 3
w = 0.7 #intertia
xr = features #dataset under which PSO needs to be performed
yr = label #dataset under which PSO needs to be performed
pp = PSO(p,d,lb,ub,it,c1,c2,w, xr, yr)

100%|█████████████████████████████████████████████████████████████████████████████████| 30/30 [51:19<00:00, 102.66s/it]


In [39]:
classifier = DecisionTreeClassifier()
# classifier2 = DecisionTreeClassifier()

In [40]:
input1 = []
y_test = label

for i in range(d):
  if pp.gbest[i] == 1:
    input1.append(features[:,i])

input1 = np.array(input1).reshape((features.shape[0],-1))

In [41]:
classifier.fit(input1, y_test)
print(input1)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [42]:
preds = classifier.predict(input1)

In [48]:
    print('Accuracy score is : ' , format(accuracy_score(y_test, preds)))
    

Accuracy Score =  0.8950107681263461
Precision score =  1.0
Recall score =  0.21686746987951808


In [ ]:
def print_metrics(y_true, preds):
    print("Accuracy Score is : ",format(accuracy_score(y_test, dt_pred)))
    print('Precision score is : ' , format(precision_score(y_test, dt_pred)))
    print('Recall score is : ' , format(recall_score(y_test, dt_pred)))
    print('F1 score is : ', format(f1_score(y_test, dt_pred)))

In [ ]:
print_metrics(y_test, dt_pred)

In [55]:
# def Predict(message):
#     message = [message]
#     message = vectorizer.transform(message).toarray()
#     print(message.shape)
#     predictedValue = classifier1.predict(message)
#     if(predictedValue == 0):
#         print("Ham")
#     else:
#         print("Spam")
    

In [36]:
# Predict(df["text"][1])
# # for i in range(1,1000):
# #     Predict(df["text"][i])